In [1]:
from sklearn.neighbors import KDTree
import numpy as np
import random
import dask.array as da
import napari

%gui qt

coord_path = 'C:/Users/cameron.trader/OneDrive - Allen Institute/Documents/Unet/0539050069-0035_r.csv'

coord = np.loadtxt(coord_path, skiprows=1, delimiter=',')

In [2]:
zarr_img_path = 'C:/Users/cameron.trader/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr'
image = da.from_zarr(zarr_img_path)

In [3]:
viewer = napari.view_image(image, name='slice')
viewer.add_points(coord[:, :2], size=15, face_color='red')

C:\Users\cameron.trader\Anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000, 3) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Points layer 'Points' at 0x166b697c2e0>

In [13]:
kdt = KDTree(coord[:,:2], leaf_size=30, metric='euclidean')

close_points = []
close_set =[]
indices_to_drop = []

close = kdt.query_radius(coord[:,:2], r=3) 
for i in close:
    if i.size > 1:
        close_points.append(list(i))
for i in close_points:
    if not i in close_set:
        close_set.append(i)
print("Close points are: ", str(close_set))
for i in close_set:
    while len(i)>1:
        index = random.randrange(len(i))
        indices_to_drop.append(i[index])
        del i[index]

print(str(len(indices_to_drop)),"points to be removed:", str(indices_to_drop))
print("These points are located at:")
np.set_printoptions(suppress=True)
for i in indices_to_drop:
    print(coord[indices_to_drop])
np.set_printoptions(suppress=False)
coord_clean = np.array([i for j, i in enumerate(coord) if j not in indices_to_drop])
coord_removed = np.array([i for j, i in enumerate(coord) if j in indices_to_drop])

print(str(coord.shape[0]-coord_clean.shape[0]), "points removed")

Close points are:  [[15172, 15173], [16419, 16420]]
2 points to be removed: [15172, 16419]
These points are located at:
[[17838.5 29552.      8. ]
 [21407.5 20171.      6. ]]
[[17838.5 29552.      8. ]
 [21407.5 20171.      6. ]]
2 points removed


In [276]:
np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_clean.csv', coord_clean, delimiter=",")

np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_removed.csv', coord_removed, delimiter=",")